In [10]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt



In [11]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#CONSTANT
BATCH_SIZE = 128
NB_EPOCH = 4
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [12]:
#convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

#normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#network
model = Sequential()
model.add(Conv2D(32,(3,3), padding="same", 
                 input_shape = (IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation("relu"))
model.add(Conv2D(32,(3,3), padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding = 'same'))
model.add(Activation("relu"))
model.add(Conv2D(64, 3, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)        18496     
__________

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [13]:
#train
model.compile(loss='categorical_crossentropy',optimizer = OPTIM,
             metrics = ['accuracy'])
model.fit(X_train,Y_train,batch_size=BATCH_SIZE,verbose = VERBOSE, 
          epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT)
score = model.evaluate(X_test,Y_test,batch_size=BATCH_SIZE,verbose = VERBOSE)

print("Test score:", score[0])
print('Test accuracy:', score[1])
#save
model_json = model.to_json()
open('cifar10_architecture.json','w').write(model_json)
model.save_weights('cifar10_weights.h5', overwrite = True)


Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 522s - loss: 1.7591 - acc: 0.3664 - val_loss: 1.4346 - val_acc: 0.4735
Epoch 2/4
40000/40000 [==============================] - 512s - loss: 1.2938 - acc: 0.5410 - val_loss: 1.2018 - val_acc: 0.5821
Epoch 3/4
40000/40000 [==============================] - 502s - loss: 1.0700 - acc: 0.6239 - val_loss: 1.0416 - val_acc: 0.6364
Epoch 4/4
10000/10000 [==============================] - 40s    
Test score: 0.893903872299
Test accuracy: 0.6859
